In [94]:
import cv2
import albumentations
import albumentations.pytorch

import pandas as pd
from hun_kfold import Run_Split
import torch

In [95]:
class Mask_Dataset(object):
	def __init__(self, transforms, name, df):
		self.transforms = transforms
		self.name = name
		self.imgs = list(
			sorted(
				os.listdir(
					f"/opt/ml/image-classification-level1-12/templates/data/train/image_all/{self.name}_image"
				)
			)
		)
		self.df = df

	def __getitem__(self, idx):
		img_path = self.df["path"][idx]
		target = self.df["label"][idx]


		img = cv2.imread(img_path)
		img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)


		if self.transforms is not None:
			augmented = self.transforms(image = img)
			img = augmented[img]

		return img, target

	def __len__(self):
		return len(self.imgs)

In [87]:
data_transform = albumentations.Compose([
    albumentations.Resize(256, 256), 
    albumentations.RandomCrop(224, 224),
    albumentations.HorizontalFlip(), # Same with transforms.RandomHorizontalFlip()
    albumentations.pytorch.transforms.ToTensorV2(),
    # albumentations.Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2))
])


In [88]:
train_path = "/opt/ml/image-classification-level1-12/templates/data/train"
train_label = pd.read_csv(os.path.join(train_path, "train_with_label.csv"))
run_split = Run_Split(os.path.join(train_path, "image_all"))
train_list, val_list = run_split.train_val_split(train_label)

device = torch.device(
	"cuda:0" if torch.cuda.is_available() else "cpu"
)  # 학습 때 GPU 사용여부 결정. Colab에서는 "런타임"->"런타임 유형 변경"에서 "GPU"를 선택할 수 있음
print(f"{device} is using!")

train_dataset = Mask_Dataset(data_transform, f"train{0}", train_list[0])
train_loader = torch.utils.data.DataLoader(
		train_dataset,
		batch_size=64,
		#  num_workers=2
	)
from pathlib import Path
next(iter(train_loader))

cuda:0 is using!


KeyError: 'img'